# Install denoiser

In [1]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following packages were automatically installed and are no longer required:
  cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-curand-10-1 cuda-curand-dev-10-1
  cuda-cusolver-10-1 cuda-cusolver-dev-10-1 cuda-cusparse-10-1
  cuda-cusparse-dev-10-1 cuda-drivers cuda-license-10-2 cuda-npp-10-1
  cuda-npp-dev-10-1 cuda-nsight-10-1 cuda-nsight-compute-10-1
  cuda-nsight-systems-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1
  cuda-nvjpeg-10-1 cuda-nvjpeg-dev-10-1 cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1
  cuda-nvvp-10-1 default-jre dkms freeglut3 freeglut3-dev
  keyboard-configuration libargon2-0 libcap2 libcryptsetup12 libcublas10
  libdevmapper1.02.1 libfontenc1 libgtk2.0-0 libgtk2.0-common libidn11
  libip4tc0 libjansson4 libnvidia-cfg1-440 libnvidia-common-430
  libnvidia-common-440 libnvidia-decode-440 libnvidia-encode-440
  libnvidia-fb

In [2]:
pip install audioread

In [0]:
%tensorflow_version 1.x

In [3]:
pip install spleeter # pip install spleeter-gpu

     |████████████████████████████████| 10.5MB 12.1MB/s 
     |████████████████████████████████| 109.2MB 27kB/s 
     |████████████████████████████████| 3.2MB 39.6MB/s 
     |████████████████████████████████| 491kB 52.9MB/s 
  Created wheel for spleeter: filename=spleeter-1.4.8-cp36-none-any.whl size=43664 sha256=354a342b7ec6aa9b362825cd5f054bf3c4f7580a8b275def2aa93380fae35320
  Stored in directory: /root/.cache/pip/wheels/eb/04/c3/0a765ad3b96a467bd3bc3a817430dd79c050b3d3b716a81771
Successfully built spleeter
  Found existing installation: pandas 0.25.3
    Uninstalling pandas-0.25.3:
      Successfully uninstalled pandas-0.25.3
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstall

In [0]:
import IPython.display as ipd

In [0]:
import audioread

In [0]:
from os import environ

In [0]:
environ["GITHUB_REPOSITORY"] = "bruce-willis/Audio-denoiser"
environ["GITHUB_RELEASE"] = "v0.1"

In [0]:
from spleeter.separator import Separator
from spleeter.audio.adapter import get_default_audio_adapter
from spleeter.audio.convertor import to_stereo

In [11]:
! wget https://raw.githubusercontent.com/bruce-willis/Audio-denoiser/develop/src/training/config/voice_config.json

--2019-12-06 12:20:31--  https://raw.githubusercontent.com/bruce-willis/Audio-denoiser/develop/src/training/config/voice_config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 890 [text/plain]
Saving to: ‘voice_config.json’

voice_config.json   100%[===================>]     890  --.-KB/s    in 0s      

2019-12-06 12:20:31 (75.7 MB/s) - ‘voice_config.json’ saved [890/890]



In [0]:
config_path = "voice_config.json"

In [0]:
separator = Separator(config_path)

In [14]:
predictor = separator._get_predictor()

INFO:spleeter:Downloading model archive https://github.com/bruce-willis/Audio-denoiser/releases/download/v0.1/voice_model.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded voice_model archive
INFO:spleeter:voice_model model file(s) extracted


INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models/voice_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8132339160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

INFO:tensorflow:Apply unet for voice_spectrogram
INFO:tensorflow:Apply unet for noise_spectrogram
INFO:tensorflow:Done calling model_fn

In [15]:
predictor

SavedModelPredictor with feed tensors {'mix_stft': <tf.Tensor 'transpose_1:0' shape=(?, 2049, 2) dtype=complex64>, 'mix_spectrogram': <tf.Tensor 'strided_slice_3:0' shape=(?, 512, 1024, 2) dtype=float32>, 'audio_id': <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=string>, 'waveform': <tf.Tensor 'Placeholder:0' shape=(?, 2) dtype=float32>} and fetch_tensors {'audio_id': <tf.Tensor 'Placeholder_1:0' shape=<unknown> dtype=string>, 'voice': <tf.Tensor 'strided_slice_13:0' shape=(?, 2) dtype=float32>, 'noise': <tf.Tensor 'strided_slice_23:0' shape=(?, 2) dtype=float32>}

In [0]:
def process_audio(filename):
    with audioread.audio_open(filename) as f:
        sample_rate = f.samplerate
    adapter = get_default_audio_adapter()
    waveform, _ = adapter.load(filename, sample_rate=sample_rate)
    if not waveform.shape[-1] == 2:
        waveform = to_stereo(waveform)
    prediction = predictor({
            'waveform': waveform,
            'audio_id': ''})
    _ = prediction.pop('audio_id')
    predicted_voice_path = f'output/voice.wav'
    adapter.save(predicted_voice_path, prediction['voice'], sample_rate)
    predicted_noise_path = f'output/noise.wav'
    adapter.save(predicted_noise_path, prediction['noise'], sample_rate)
    
    print(f"Original {filename} audio")
    ipd.display(ipd.Audio(filename, rate=sample_rate))
    print("Cleaned voice")
    ipd.display(ipd.Audio(predicted_voice_path, rate=sample_rate))
    print("Separated noise")
    ipd.display(ipd.Audio(predicted_noise_path, rate=sample_rate))

In [26]:
! wget https://github.com/bruce-willis/Audio-denoiser/raw/develop/src/training/p257_331.wav

--2019-12-06 12:23:26--  https://github.com/bruce-willis/Audio-denoiser/raw/develop/src/training/p257_331.wav
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bruce-willis/Audio-denoiser/develop/src/training/p257_331.wav [following]
--2019-12-06 12:23:27--  https://raw.githubusercontent.com/bruce-willis/Audio-denoiser/develop/src/training/p257_331.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199386 (195K) [audio/wav]
Saving to: ‘p257_331.wav.2’

p257_331.wav.2      100%[===================>] 194.71K  --.-KB/s    in 0.02s   

2019-12-06 12:23:27 (8.50 MB/s) - ‘p257_331.wav.2’ saved [199386/1

In [27]:
process_audio("p257_331.wav")

INFO:spleeter:File output/voice.wav written
INFO:spleeter:File output/noise.wav written


Original p257_331.wav audio


Cleaned voice


Separated noise
